In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer,LlamaConfig
from datasets import load_dataset
import bitsandbytes as bnb
# config = LlamaConfig.()
tokenizer = LlamaTokenizer.from_pretrained("/data/zhangyi/hf_model/Llama2-7b-chat-hf")
model  = LlamaForCausalLM.from_pretrained("/data/zhangyi/hf_model/Llama2-7b-chat-hf",device_map="cuda:0"
                                        )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# peft
# model.print_trainable_parameters()
for name, layer in model.named_children():
    print(f'Layer: {name} | Details: {layer}')

Layer: model | Details: LlamaModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
        (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
        (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm()
      (post_attention_layernorm): LlamaRMSNorm()
    )
  )
  (norm): LlamaRMSNorm()
)
Layer: lm_head | Details: Linear(in_features=4096, out_features=3

In [4]:
for name,parameter in list(model.named_parameters())[:5]:
    print(name) 
    print(parameter.shape)
    print(parameter.requires_grad)

model.embed_tokens.weight
torch.Size([32000, 4096])
True
model.layers.0.self_attn.q_proj.weight
torch.Size([4096, 4096])
True
model.layers.0.self_attn.k_proj.weight
torch.Size([4096, 4096])
True
model.layers.0.self_attn.v_proj.weight
torch.Size([4096, 4096])
True
model.layers.0.self_attn.o_proj.weight
torch.Size([4096, 4096])
True


In [14]:
# index = 0
# for i in model.parameters():
#     index +=1
# index
total_params = sum(p.numel() for p in model.parameters())

print(f"Total parameters: {total_params}")
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Total parameters: 6758404096
Trainable parameters: 19988480


# peft load 

In [8]:
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType, PeftModel
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                        inference_mode=False,
                        r=8 ,
                        lora_alpha=32,
                        lora_dropout=0.1 ,
                        target_modules=['k_proj', 'v_proj', 'q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'])
peft_model = get_peft_model(model, peft_config)

In [13]:

peft_model.print_trainable_parameters()
-(6738415616  - 6758404096)

trainable params: 19,988,480 || all params: 6,758,404,096 || trainable%: 0.2957573965106688


19988480

In [10]:
for name, layer in peft_model.named_children():
    print(f'Layer: {name} | Details: {layer}')

Layer: base_model | Details: LoraModel(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): lora.Linear(
              (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=8, out_features=4096, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (k_proj): lora.Linear(
              (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
              (lora_dro